In [ ]:
import altair as alt
import numpy as np
import os
import pandas as pd

In [ ]:
tz = "America/Los_Angeles"

In [ ]:
donations = pd.read_csv("data/donations.csv", parse_dates=["timestamp"])
donations["timestamp"] = pd.to_datetime(donations["timestamp"], utc=True).dt.tz_convert(tz)
donations.tail(10)

In [ ]:
tests = pd.read_csv("data/tests.csv")
tests["timestamp"] = pd.to_datetime(tests["timestamp"], utc=True).dt.tz_convert(tz)
tests.tail(10)

In [ ]:
events = pd.merge(tests, donations, how="outer", on=["timestamp"])

In [ ]:
def plot_test(events, test_name, reference_range):
    timeline = alt.Chart(events).encode(
        alt.X("timestamp:T", axis=alt.Axis(title=None))
    ).properties(
        width = 700,
        height = 150
    )

    values_chart = timeline.mark_point(opacity=0.8).encode(
        alt.Y("value:Q", scale = alt.Scale(zero=True), title=test_name),
        tooltip=[alt.Tooltip("timestamp:T", format="%x %X"), "value:Q"]
    ).transform_filter(
        f'datum.test == "{test_name}"'
    )  

    donations_chart = timeline.mark_rule(color="black", opacity=0.8, strokeDash=[1, 1]).encode(
        tooltip=["timestamp:T"],
        size=alt.value(2)
    ).transform_filter(
        alt.datum.quantity > 0
    )

    reference_range_chart = alt.Chart().mark_rect(opacity=0.1, color="gray").encode(
        alt.Y("lower:Q"),
        alt.Y2("upper:Q")
    ).transform_calculate(lower=str(reference_range[0]), upper=str(reference_range[1]))

    return alt.layer(reference_range_chart, donations_chart, values_chart)

In [ ]:
iron_total = plot_test(events, "Iron, Total (ug/dL)", (50, 180))
iron_binding_capacity = plot_test(events, "Iron Binding Capacity (ug/dL)", (250, 425))
iron_saturation = plot_test(events, "Iron Saturation (%)", (20, 48))
ferritin = plot_test(events, "Ferritin (ng/mL)", (38, 380))
hg = plot_test(events, "Hemoglobin (g/dL)", (13.2, 17.1))
alt.vconcat(iron_total, iron_binding_capacity, ferritin, hg).resolve_scale(
    x="shared"
)